# SetUp

I will install, download, and import all necessary packages for the tasks in the following. 

## Installations

In order to run the following codes, it might be required to install all these packes first.

In [1]:
# !pip3 install pdfminer3
# !pip3 install contractions
# !pip3 install spacy
# !pip3 install unicodedata
# !pip3 install nltk
# !pip3 install numpy
# !pip3 install string
# !pip3 install pandas
# !pip3 install gensim
# !pip3 install sklearn
# !pip3 install matplotlib
# !pip3 install plotly
# !pip3 install torch
# !pip3 install torchvision
# !pip install --upgrade gensim
# !pip install 'umap-learn==0.3.10'

In [2]:

# pdb.set_trace()

In [3]:
import collections
import pdb
import re
import unicodedata

import nltk
import spacy
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer, sent_tokenize, word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer

nltk.download('punkt')

import io
import itertools
import shutil
import string

import contractions
import gensim.downloader as api
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import torch
import torchvision
from gensim.models import Word2Vec
from pdfminer3.converter import PDFPageAggregator, TextConverter
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer3.pdfpage import PDFPage
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
from spacy.lang.en import English
from transformers import pipeline

import sys
sys.path.append(r"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area Bulldozer\Sandbox")
from nlp_functions import (classifier, remove_colons, remove_digits, remove_n,
                           remove_redundant_whitespaces,
                           remove_strange_characters, remove_stripes,
                           text_loader)
import itertools
from nlp_functions import text_loader, classifier, remove_strange_characters, remove_n, remove_colons, remove_stripes, remove_redundant_whitespaces, remove_digits
import torchvision
import torch
import plotly.express as px
from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import matplotlib
from sklearn.manifold import TSNE
import gensim.downloader as api
from spacy.lang.en import English
import pandas as pd
from gensim.models import Word2Vec
import os
import shutil
import io
from pdfminer3.converter import TextConverter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfpage import PDFPage
from pdfminer3.layout import LAParams, LTTextBox
import contractions
import string
import numpy as np
import spacy
import unicodedata
import re

import collections

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from statistics import mean


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daveb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daveb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daveb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Loading the text

In [4]:
def pdf_loader(company_name):
    company_name = company_name[:-4]
    source = 'D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area\Data\Sustainability Reports\To Do'
    path = rf"{source}\{company_name}.pdf"

    text = text_loader(path)

    return text, company_name, path, source

# Preprosessing

In this chapter, I will apply the methods I learned in the previous exercise. I will limit it only to the ones I feel most needed/suitable.

### Remove Digits

In [5]:
def preprocessing(text):
    text = re.sub(r'\d+', '', text)

    sentences  = []
    for s in sent_tokenize(text):
        sentences.append(s)

    df = pd.DataFrame(sentences, columns=['Sentences'])

    df['Alt_Text'] = df['Sentences'].apply(lambda x: remove_strange_characters(x))
    df['Alt_Text'] = df['Alt_Text'].apply(lambda x: remove_n(x))
    df['Alt_Text'] = df['Alt_Text'].apply(lambda x: remove_stripes(x))
    df['Alt_Text'] = df['Alt_Text'].apply(lambda x: remove_redundant_whitespaces(x))

    return df


    


# Part 1: Cosine Similarity Scoring Sentcne by Sentence

This part we will different scoring methods and try to apply them. The similarity score to a pregiven text will be the major idea atm.

In [6]:
# bringing this function to the function class

def similarity_function(text_1, text_2):
    sentences = [text_1, text_2]

    # !pip install sentence_transformers
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    sentence_embeddings = model.encode(sentences)
    sentence_embeddings.shape
    return cosine_similarity(
        [sentence_embeddings[0]],
        sentence_embeddings[1:]
        )






In [7]:
def scoring(report_df):
    wiki_text = pd.read_csv(r"G:\Meine Ablage\HSLU\Master\Master Thesis\Coding Area\Data\wiki_artikel.csv")
    result_cosine_similarity = []
    wiki_text_list = wiki_text["Text"].to_list()
    wiki_labels_list = wiki_text["Label"].to_list()
    report_text_list = report_df["Text"].to_list()

    for w_text, r_text in zip(wiki_text_list, report_text_list):
        pre_result = (similarity_function(w_text, r_text))
        result_cosine_similarity.append(pre_result[0][0])

    data_tuples = list(zip(wiki_labels_list, result_cosine_similarity))
    result_df = pd.DataFrame(data_tuples, columns=['Label','Score'])
    return result_df

### Wikipedia Summarized by BERT

In [8]:
wiki_text = pd.read_csv(
    r"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area\Data\wiki_artikel_fulltext.csv", index_col=0)
wiki_text_list = wiki_text["Text"].tolist()


from transformers import pipeline
wiki_summarized_list = []
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

for label in wiki_text_list:
    label = label[0:1023]
    label_summarized = (summarizer(label, max_length=130, min_length=100, do_sample=False))
    wiki_summarized_list.append(label_summarized)


wiki_summarized_list_final = []
for element in wiki_summarized_list:
    new_element = element[0]["summary_text"]
    wiki_summarized_list_final.append(new_element)


wiki_text["Summarized"] = wiki_summarized_list_final
wiki_summarized = wiki_text["Summarized"]

In [9]:
len(wiki_summarized[4].split(" "))

80

In [10]:
wiki_summarized[4]

"Labour laws are those that mediate the relationship between workers, employing entities, trade unions, and the government. Collective labour law relates to the tripartite relationship between employee, employer, and union.Individual labour law concerns employees' rights at work also through the contract for work. Employment standards are social norms for the minimum socially acceptable conditions under which employees or contractors are allowed to work. Government agencies (such as the former US Employment Standards Administration) enforce labour law (legislature, regulatory, or judicial)."

In [11]:
wiki_text = pd.read_csv(
    r"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area\Data\wiki_artikel_fulltext.csv", index_col=0)
wiki_text_list = wiki_text["Text"].tolist()


from transformers import pipeline
wiki_summarized_list = []
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

for label in wiki_text_list:
    label = label[0:1023]
    label_summarized = (summarizer(label, max_length=130, min_length=100, do_sample=False))
    wiki_summarized_list.append(label_summarized)


wiki_summarized_list_final = []
for element in wiki_summarized_list:
    new_element = element[0]["summary_text"]
    wiki_summarized_list_final.append(new_element)


wiki_text["Summarized"] = wiki_summarized_list_final
wiki_summarized = wiki_text["Summarized"]

In [12]:
wiki_summarized[4]

"Labour laws are those that mediate the relationship between workers, employing entities, trade unions, and the government. Collective labour law relates to the tripartite relationship between employee, employer, and union.Individual labour law concerns employees' rights at work also through the contract for work. Employment standards are social norms for the minimum socially acceptable conditions under which employees or contractors are allowed to work. Government agencies (such as the former US Employment Standards Administration) enforce labour law (legislature, regulatory, or judicial)."

### Needed Functions for the TF-IDF

In [13]:
def preprocessing_tf(text):
    # mapping = str.maketrans('', '', string.digits)
    # text = text.translate(mapping)

    text = remove_strange_characters(text)
    text = re.sub(r'\d+', '', text)


    text = remove_n(text)

    text = remove_colons(text)

    text = text.replace(r" .",".")
    text = text.replace(r"..",".")
    text = text.replace(r"...",".")

    text = remove_stripes(text)

    text = remove_redundant_whitespaces(text)
    return text


nltk.download('omw-1.4')
nltk.download('wordnet')

def lemmatize_words(text):

    lemmatizer = WordNetLemmatizer()
    text = word_tokenize(text)

    lemma_list = []

    for word in text:
        lemma_word = lemmatizer.lemmatize(word)
        lemma_list.append(lemma_word)

    lemma_text = ' '.join(lemma_list)


    return lemma_text



def remove_small_tokens(prep_text):
    prep_text_before = prep_text
    # print(f"Text before cleaning:  {len(prep_text_before)}")
    # print(type(prep_text_before))
    # prep_text_before = [prep_text_before]
    prep_text_before = word_tokenize(prep_text_before)
    # print(prep_text_before)

    for word in prep_text_before:
        if len(word) <= 3:
            prep_text_before.remove(word)
    # print(f"Text after cleaning:  {len(prep_text_before)}")
    prep_text_list = ' '.join(prep_text_before)
    # print(type(prep_text_list))


    return prep_text_list

# BOW

def get_top_n_words(corpus, n=None):
    corpus=[corpus]

    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    top_n_words_df = pd.DataFrame(words_freq, columns =['Word', 'Count'])

    return top_n_words_df.loc[:(n-1)]


def get_tf_idf(text, n=None):
    text = re.findall(r'(?:\d[.]|[^.])*(?:[.]|$)', text)
    tfIdfVectorizer=TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform(text)
    df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
    df = df.sort_values('TF-IDF', ascending=False)
    return (df[:(n)])



[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\daveb\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\daveb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Let it all run

In [14]:
def mean_cosine_calculator(cosine_results):
    non_tensor_list = []
    for element in cosine_results:
        string_element = str(element[0])
        non_tensor_list.append(float(string_element[8:14]))

    non_tensor_list

    after_threshold_sust = []
    threshold = 0.08

    for cosine in non_tensor_list:
        if cosine > threshold:
            after_threshold_sust.append(cosine)


    mean_result = round(mean(after_threshold_sust), 3)
    return mean_result

In [15]:
list_of_companies = os.listdir(
    r"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area\Data\Sustainability Reports\To Do")
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
from statistics import mean

for company in list_of_companies:
    ## BOW / TF-IDF
    print(f"We process now {company}")
    text_1, company_name, path, source = pdf_loader(company)
    prep_text = preprocessing_tf(text_1)
    lemma_words = lemmatize_words(prep_text)
    text_cleaned = remove_small_tokens(lemma_words)
    top_n_words = get_top_n_words(text_cleaned, 20)
    top_n_words = list(top_n_words.itertuples(index=False, name=None))
    top_n_words
    tf_idf = get_tf_idf(text_cleaned, 20)
    tf_idf = tf_idf.reset_index()
    tf_idf = tf_idf.rename(columns={"index": "Word"})
    idf_list = list(tf_idf.itertuples(index=False, name=None))
    idf_list
    data = [(company_name, idf_list, top_n_words)]
    df1 = pd.DataFrame(data, columns = ["Company Name", "TF-IDF", "Top N Words"])
    # print(df1)
    company_name_new = f"{df1['Top N Words'][0][0][0]}_{df1['Top N Words'][0][1][0]}_{df1['Top N Words'][0][2][0]}"
    print(company_name_new)

    df_1 = preprocessing(text_1)
    text_list = df_1["Alt_Text"].tolist()
    new_text_list = []
    print(len(text_list))
    for element in text_list:
        if len(element) < 15:
            text_list.remove(element)
        if len(element) > 15:
            new_text_list.append(element)
    print(len(text_list))
    print(len(new_text_list))

    cos_sim_sust_results = []
    cos_sim_human_rights_results = []
    cos_sim_fraud_results = []
    cos_sim_social_issues_result = []
    cos_sim_social_labour_law_results = []

    for sentence in new_text_list:
        emb1 = model.encode(sentence)
        emb2 = model.encode(wiki_summarized[0])
        cos_sim_sust = util.cos_sim(emb1, emb2)
        cos_sim_sust_results.append(cos_sim_sust)

        emb1 = model.encode(sentence)
        emb2 = model.encode(wiki_summarized[1])
        cos_sim_human_rights = util.cos_sim(emb1, emb2)
        cos_sim_human_rights_results.append(cos_sim_human_rights)

        emb1 = model.encode(sentence)
        emb2 = model.encode(wiki_summarized[2])
        cos_sim_fraud = util.cos_sim(emb1, emb2)
        cos_sim_fraud_results.append(cos_sim_fraud)

        emb1 = model.encode(sentence)
        emb2 = model.encode(wiki_summarized[3])
        cos_sim_social_issues = util.cos_sim(emb1, emb2)
        cos_sim_social_issues_result.append(cos_sim_social_issues)

        emb1 = model.encode(sentence)
        emb2 = model.encode(wiki_summarized[4])
        cos_sim_social_labour_law = util.cos_sim(emb1, emb2)
        cos_sim_social_labour_law_results.append(cos_sim_social_labour_law)

    all_cosine_results = [cos_sim_sust_results, cos_sim_human_rights_results, cos_sim_fraud_results, cos_sim_social_issues_result, cos_sim_social_labour_law_results]
        

    # cosine_mean = []
    # for label in all_cosine_results:
    #     mean_label = mean_cosine_calculator(label)
    #     cosine_mean.append(mean_label)
    # cosine_mean

    labels = wiki_text["Label"]

    data = {'Label':labels,'Cosine Similarity Mean':all_cosine_results}
    final_df = pd.DataFrame(data)
    # print(df1)
    final_df.to_csv(fr"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area\Data\Results Bulldozer\Sentence by Sentence Cosine Similarity raw\{company_name_new}_cosine_raw.csv")
    print("SAFED")

We process now 10_BCGB20_SustainabilityReport_E_Web.pdf


c:\Users\daveb\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


cid_burckhardt_compression
268
259
250
SAFED
We process now 1212.pdf


c:\Users\daveb\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


bell_food_group
711
700
688
SAFED
We process now 1313.pdf


...

sustainability_bank_report
159
154
149
SAFED
We process now wuerth_nachhaltigkeitsbericht_2019_1.pdf


c:\Users\daveb\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


die_würth_der
1625
1537
1447
SAFED
We process now Zegna-responsibility.pdf


c:\Users\daveb\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


group_zegna_supplier
304
292
281
SAFED
